In [ ]:
import requests
import time
import json
from pprint import pprint

base_url = 'http://127.0.0.1:8001'
api_key = ''

algorithm_name = 'Pinnacle Export'
api_dicom_location = '{0}/api/dicomlocation'.format(base_url)
api_dataset = '{0}/api/dataset'.format(base_url)
api_dataset_ready = '{0}/api/dataset/ready'.format(base_url)
api_data_object = '{0}/api/dataobject'.format(base_url)
api_trigger = '{0}/api/trigger'.format(base_url)
api_algorithm = '{0}/api/algorithm'.format(base_url)
api_download = '{0}/api/dataobject/download'.format(base_url)

In [ ]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        pprint(a)
        if a['name'] == algorithm_name:
            algorithm = a
    print("")
    print("Look's Good!")
else:
    print("Oops, something went wrong. Ensure the service is running at the base_url configured and that the API Key has been generated and set in api_key.")

In [ ]:
# Check if we already have the Orthanc Dicom Location, if not lets add it
orthanc_location = None
r = requests.get(api_dicom_location, headers={'API_KEY': api_key})
for d in r.json():
    if d['name'] == 'Orthanc':
        orthanc_location = d
        break
        
if not orthanc_location:
    # If we didn't find the Orthanc location then lets add it now
    data = {'name': 'Orthanc',
           'host': '127.0.0.1',
           'port': 4242,
           'ae_title': 'ORTHANC'}
    r = requests.post(api_dicom_location, headers={'API_KEY': api_key}, data=data)
    
    if r.status_code >= 200 and r.status_code < 300:
        print('Added Orthanc Location')
        orthanc_location = r.json()
        
pprint(orthanc_location)

In [ ]:
# Create a new Dataset
dataset = None
data = {'to_dicom_location': orthanc_location['id']}
data = {}
r = requests.post(api_dataset, headers={'API_KEY': api_key}, data=data)
if r.status_code >= 200:
        dataset = r.json()
        
pprint(dataset)

In [ ]:
# Add a Pinnacle Tar archive to the dataset
path_to_tar = f'../tests/data/pinnacle/test_pinnacle_16.0.tar.gz'
data_object = None

# Get the Series UID of this Data Object
with open(path_to_tar,'rb') as file:
    
    meta_data = {
        "patient_path": "Institution_2812/Mount_0/Patient_16218", # The path to the patient folder within the tar archive
        "plan_name": "Plan_2", # The name of the plan to export in pinnacle (optional, default use first),
        "trial": "", # The name of the trial to export in the plan (optional, default: use FINAL or first),
        'some_test_data': "TESTING 1234"
    }

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            'meta_data': json.dumps(meta_data),
            'file_name': 'test_pinnacle_16.0.tar.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            data_object = r.json()
        
pprint(data_object)

In [ ]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        if algorithm_name in a['name']:
            algorithm = a
        print(a)

In [ ]:
# Trigger the algorithm with our dataset containing the data object
data={'dataset': dataset['id'],
     'algorithm': algorithm['name']}
r = requests.post(api_trigger, headers={'API_KEY': api_key}, data=data)

if r.status_code == 200:
    # Poll the URL given to determine the progress of the task
    poll_url = '{0}{1}'.format(base_url, r.json()['poll'])
    
    while(1):
        r = requests.get(poll_url, headers={'API_KEY': api_key})
        status = r.json()
        print(status)

        if status['state'] == 'SUCCESS' or status['state'] == 'FAILURE':
            break

        time.sleep(1)
else:
    print(r.json())
    
print('Algorithm Processing Complete')

In [ ]:
# Take a look at the dataset objects output
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)